```{contents}
```
## Secret Management

Secret management in LangGraph refers to the **secure storage, access, rotation, and usage of sensitive credentials** (API keys, tokens, passwords, certificates) across graph execution, agents, tools, and environments—**without exposing them in code, logs, state, or memory**.

---

### **1. Why Secret Management Matters in LangGraph**

LangGraph workflows interact with:

* LLM APIs (OpenAI, Anthropic, etc.)
* Databases
* Vector stores
* Internal services
* Cloud resources

Without proper secret management, systems risk:

| Risk                  | Impact             |
| --------------------- | ------------------ |
| Credential leakage    | Security breach    |
| Model abuse           | Cost explosion     |
| Compliance violations | Legal risk         |
| System compromise     | Production failure |

---

### **2. Design Principles**

| Principle             | Description                       |
| --------------------- | --------------------------------- |
| No secrets in code    | Never hardcode keys               |
| No secrets in state   | State must be serializable & safe |
| Environment isolation | Dev / Staging / Prod separation   |
| Least privilege       | Limit access scope                |
| Rotation              | Regular secret updates            |
| Auditability          | Track access & usage              |

---

### **3. Where Secrets Live in LangGraph Systems**

LangGraph itself does **not store secrets**.
It consumes secrets from **secure external sources**.

| Layer      | Storage                   |
| ---------- | ------------------------- |
| Local dev  | `.env` files              |
| Server     | OS environment variables  |
| Cloud      | Secret Manager / Vault    |
| Containers | Kubernetes Secrets        |
| CI/CD      | Secure pipeline variables |

---

### **4. Standard Secret Flow in LangGraph**

```
Secret Store → Environment → Tool/LLM Client → LangGraph Node
```

**Secrets are injected only at runtime.**

---

### **5. Accessing Secrets in LangGraph Nodes**

```python
import os

def llm_node(state):
    api_key = os.environ["OPENAI_API_KEY"]
    ...
```

**Never** put secrets in:

* Graph state
* Node return values
* Logs
* Checkpoints

---

### **6. Using Secrets with Tools**

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)
```

---

### **7. Production-Grade Secret Backends**

| Platform   | Solution        |
| ---------- | --------------- |
| AWS        | Secrets Manager |
| Azure      | Key Vault       |
| GCP        | Secret Manager  |
| Kubernetes | K8s Secrets     |
| Enterprise | HashiCorp Vault |

Example (AWS):

```python
import boto3

client = boto3.client("secretsmanager")
secret = client.get_secret_value(SecretId="openai_key")["SecretString"]
```

---

### **8. Preventing Secret Leakage**

| Control            | How                       |
| ------------------ | ------------------------- |
| Log filtering      | Never log env             |
| State sanitization | Validate outputs          |
| Access control     | Node-level permissions    |
| Redaction          | Mask tokens               |
| Memory exclusion   | Do not store in vector DB |

---

### **9. Secret Rotation Strategy**

1. Store versioned secrets
2. Rotate on schedule
3. Update environment
4. Restart LangGraph workers
5. Verify health

---

### **10. Secure Deployment Pattern**

```
Kubernetes Pod
   |
Env Injection from K8s Secrets
   |
LangGraph Runtime
   |
LLM & Tools
```

---

### **11. Compliance & Governance**

| Requirement       | Implementation            |
| ----------------- | ------------------------- |
| Access audit      | Central secret manager    |
| Encryption        | At rest + in transit      |
| RBAC              | Who can read which secret |
| Key expiration    | Mandatory rotation        |
| Incident response | Immediate revoke          |

---

### **12. Anti-Patterns (Never Do This)**

```python
api_key = "sk-123..."   # ❌
state["api_key"] = os.environ["OPENAI_API_KEY"]  # ❌
print(os.environ)  # ❌
```

---

### **13. Minimal Secure Example**

```python
from langgraph.graph import StateGraph
import os

def agent_node(state):
    llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])
    ...
```

---

### **14. Mental Model**

> **LangGraph orchestrates behavior.
> Your platform governs secrets.**

LangGraph remains **stateless and secret-agnostic**, while your infrastructure ensures **secure execution**.


### Demonstration

In [2]:
# Secure, production-style LangGraph setup with secret management (single cell demo)

import os
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

# ---- 1. Assume secret is injected by environment (not in code!) ----
# Example: export OPENAI_API_KEY=sk-...

assert "OPENAI_API_KEY" in os.environ, "Missing secret in environment"

# ---- 2. Define State ----
class State(TypedDict):
    question: str
    answer: str

# ---- 3. Define Node that consumes secret safely ----
def llm_node(state: State):
    llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])
    response = llm.invoke(state["question"])
    return {"answer": response.content}

# ---- 4. Build Graph ----
builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.set_entry_point("llm")
builder.add_edge("llm", END)

graph = builder.compile()

# ---- 5. Execute (no secret ever enters state or logs) ----
result = graph.invoke({"question": "Explain gradient descent in one sentence."})
print(result["answer"])

Gradient descent is an iterative optimization algorithm used to minimize the error or loss function of a machine learning model by adjusting the model's parameters in the direction of steepest descent of the gradient.
